In [1]:
from P1_dataloader import P1_Dataset

from typing import Dict, Tuple
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from P1_model import *
from glob import glob

In [2]:
import torch
import numpy as np
import random

# 設置random seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # 確保GPU上的隨機性也固定
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 如果使用多個GPU
        torch.backends.cudnn.deterministic = True  # 保證運算可重現
        torch.backends.cudnn.benchmark = False

set_seed(42)  # 固定隨機種子為42

In [3]:
device = 'cuda:2'
ddpm = DDPM(nn_model=ContextUnet(in_channels=3, n_feat=256, n_classes=12), betas=(1e-4, 0.02), n_T=500, device=device, drop_prob=0.1)
ddpm.load_state_dict(torch.load('./output/P1_v1/model_99.pt', map_location="cuda"))
ddpm = ddpm.to(device)

/tmp/ipykernel_3126381/1640066420.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ddpm.load_state_dict(torch.load('./output/P1_v1/model_99.pt', map_location="cuda"))


In [8]:
ddpm.eval()
import os

with torch.no_grad():
    
    row_list = [] 
    for iter_num in range(10):
        x_gen, x_gen_store = ddpm.sample(c_1 = 0, n_sample=10, size=(3, 28, 28), device=device, guide_w=2)
        
        if iter_num == 0:
            zero_store_0 = x_gen_store[:, 0]
        
        column_list = []  # 創建一個列表來保存要連接的列張量
        
        for i in range(10):
            column_list.append(x_gen[i])  # 添加列張量到列表中
        
        column = torch.cat(column_list, dim=1)
        row_list.append(column)
        
    rows = torch.cat(row_list, dim=2)

    save_image(rows, 'result_0.png')
    
    row_list = [] 
    for iter_num in range(10):
        x_gen, x_gen_store = ddpm.sample(c_1 = 1, n_sample=10, size=(3, 28, 28), device=device, guide_w=2)
        
        if iter_num == 0:
            zero_store_1 = x_gen_store[:, 0]
        
        column_list = []  # 創建一個列表來保存要連接的列張量
        
        for i in range(10):
            column_list.append(x_gen[i])  # 添加列張量到列表中
        
        column = torch.cat(column_list, dim=1)
        row_list.append(column)
        
    rows = torch.cat(row_list, dim=2)

    save_image(rows, 'result_1.png')

In [16]:
zero_store_0.shape
selected_idx = [0, 4, 9, 14, 19, 20, 21, 22, 23, 31]
step = [0, 100, 200, 300, 400, 420, 440, 460, 480, 500]
for idx in range(len(selected_idx)):
    save_image(torch.Tensor(zero_store_0[selected_idx[idx]]), './results/P1_report/0_{:03d}.png'.format(step[idx]))
    save_image(torch.Tensor(zero_store_1[selected_idx[idx]]), './results/P1_report/1_{:03d}.png'.format(step[idx]))